In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import wooldridge as woo
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import preprocessing
import seaborn as sns

# Week 3 Exercise

## Question 1:
We want to choose the right specification to create our regression model. For this exercise we will be using the **hprice1** dataset from wooldridge. We will be fitting various models to predict the **price** of a house, starting first using **sqrft** as the predictor. 

For the first step I would like you to fit several different models:
- Linear Model
- Quadratic Model

Compare the out-of-sample performance by fitting yoru model first on a training set and then calculating the root mean squared error (RMSE) over a testing set where:

The formula for Root Mean Square Error (RMSE) is given by:

$$
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2}
$$

Where:
- $n$ is the number of observations,
- $y_i$ is the actual value for the \(i\)-th observation,
- $\hat{y}_i$ is the predicted value for the \(i\)-th observation

The test set should be 30% of your data. **Which model gives the better performance on the test set?**



Note that in order to fit a quadratic model you would use a formula that takes the following form
y ~ x + I(X**2)

In [ ]:
data = woo.data('hprice1')
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train, test = train_test_split(___, test_size= __)

In [ ]:
# Fit the linear model
model = smf.ols('___ ~ ___', train).fit()
model.summary()

In [ ]:
# get the performance on the test set
predictions = model.predict(___)

# Calculate the RMSE - the code below is a start, but you need to follow the RMSE formula
test['price'] - ___

In [ ]:
# Fit the linear model
model = smf.ols('___ ~ ___', train).fit()
model.summary()

In [ ]:
# get the performance on the test set
predictions = model.predict(___)

# Calculate the RMSE - the code below is a start, but you need to follow the RMSE formula
test['price'] - ___

## Question 2: What is the confidence interval around your estimates of the coefficients for each predictor in the better model? (Check the summary)

## Question 3: We want to generate bootstrap confidence intervals around your estimate of the sqrft variable for the quadratic mode using the following steps:

- **Step 1:** Set a number of bootstrap iterations (e.g., 1000). This is how many times you will resample and refit the model.

- **Step 2:** For each iteration:
  - Randomly sample your data **with replacement**, so the new sample is the same size as the original dataset.
  - Fit an OLS regression model using the resampled data, with `price` as the dependent variable and `sqrft` (and its square) as the independent variable.
  - Record the estimated coefficient for `sqrft` from the fitted model.

- **Step 3:** After running all iterations, you will have a collection of estimated `sqrft` coefficients (one from each bootstrap sample).

- **Step 4:** To calculate the 95% confidence interval for the `sqrft` coefficient:
  - Sort the bootstrap estimates for the `sqrft` coefficient.
  - Identify the 2.5th percentile and the 97.5th percentile of these sorted estimates. These percentiles represent the lower and upper bounds of the 95% bootstrap confidence interval.

- **Step 5:** Report the 95% confidence interval for the `sqrft` coefficient.

**How do the bootstrap intervals compare to teh ones from the model summary?**

In [ ]:
nboot = ___
params = []
for i in range(nboot):
    # sample with replacement with length = n samples
    temp = data.sample(len(data), ___)

    # estimate model
    temp_model = smf.ols('___ ~ ___', temp).fit()

    #extract desired metrics
    params.append(temp_model.params['sqrft'])

params = pd.Series(params)

In [ ]:
# Lower bound 
print(params.quantile(___))

# Upper Bound 
print(params.quantile(___))

In [ ]:
params.hist()

Bonus Question: Below I add an unsual value to the dataset, what happens to the bootsrap estimates when this datapoint is added?
- What does this tell you about the purpose of bootstrapping?

In [ ]:
case = data[['price', 'sqrft']].copy().reset_index(drop = True)

# adding a highly influention observation tp the data
new = pd.DataFrame({'price': 200, 'sqrft': 5000}, index = [88])
case = pd.concat([case, new])

model_inf = smf.ols('price ~ sqrft + I(sqrft**2)', case).fit()

# we can see the point on our unusual observations plot where the diameter is equal to the "cooks distance"
import statsmodels.api as sm
sm.graphics.influence_plot(model_inf, ax = ax, criterion = 'cooks')

In [ ]:
case = data[['price', 'sqrft']].copy().reset_index(drop = True)


new = pd.DataFrame({'price': 200, 'sqrft': 5000}, index = [88])
case = pd.concat([case, new])


nboot = ___
params = []
for i in range(nboot):
    # sample with replacement with length = n samples
    temp = case.sample(len(case), ___)

    # estimate model
    temp_model = smf.ols('___ ~ ___', temp).fit()

    #extract desired metrics
    params.append(temp_model.params['sqrft'])

params = pd.Series(params)

In [ ]:
params.hist()

## Question 4

Below I give you starting code to generate a 95% prediction and confidence interval using the model you chose as the best in the previous question. 

- What is the intepretation of the confidence interval here? What about the prediction interval? (Hint: Note the obs_ vs mean_ before the variable names)

- What percentage of the observed values fall within the prediction interval?

In [ ]:
# here we refit the model over the entire data set
model = smf.ols('___ ~ ___', data).fit()

In [ ]:
predictions = model.get_prediction(data)                       # generate predictions
intervals = predictions.summary_frame(alpha = ___)       # compute 95% intervals
intervals

In [ ]:
# You should use boolean indexing to find the proportion of observations from the data that fall into the CI
((___ < data['price'] ) & (data['price'] < ___)).mean()

In [ ]:
x_var = 'sqrft'
y_var = 'price'

store = pd.DataFrame({x_var:np.linspace(data[x_var].min(), data[x_var].max(), 1000)})
predictions = model.get_prediction(store)                       # generate predictions
intervals = predictions.summary_frame(alpha = 0.05)       # compute 95% intervals

# Extracting estimated intervals
store['point_est'] = intervals['mean']     # point estimates
store[['ci_lower', 'ci_upper']] = intervals[['mean_ci_lower', 'mean_ci_upper']]   # lower and upper CONFIDENCE intervals
store[['pi_lower', 'pi_upper']] = intervals[['obs_ci_lower', 'obs_ci_upper']]     # lower and upper PREDICTION intervals

# Plotting
plt.figure(figsize = (10, 5))
plt.scatter(x=data[x_var], y=data[y_var], color='black', alpha = .5)
plt.plot(store[x_var], store['point_est'], color='blue', label='Fitted line')
plt.fill_between(store[x_var], store['ci_lower'], store['ci_upper'], color='royalblue', alpha=0.2, label='Confidence Interval')
plt.fill_between(store[x_var], store['pi_lower'], store['pi_upper'], color='lightsteelblue', alpha=0.2, label='Prediction Interval')
plt.ylabel(y_var)
plt.xlabel(x_var)
plt.legend()
plt.tight_layout()

## Question 5: We will now try to fit a log linear model to the dataset (hint: you will need to use np.log() in the statsmodels formula). Use the formula:
$$log(price) = \beta_1 + \beta_2 sqrft + e$$

- What is the interpretation of the coefficient sqrft for this regression equation?
- Does this regression appear to be an improvement on the linear-linear model? How can we tell? (Hint: consider the properties of the regression equation we were interested in improving using a log transform)

In [ ]:
# modify the following code
model = smf.ols('__~__', data).fit()
model.summary()

In [ ]:
model.summary()

## Question 6: Now suppose we would like to investigate whether a power transform on the dependent variable price would be appropriate. 
Recall the box cox transform takes the form:
$$
T_{BC}(x,\lambda) = x^{(\lambda)}=\begin{cases}
\frac{x^\lambda - 1}{\lambda} \phantom- \text{ when }\lambda \not= 0, \\
\ln(x) \phantom- \text{ when }\lambda = 0.
\end{cases}
$$

Where $\lambda = 0$ is simply a special case of the transform where it is equivalent to a log transform. In teh case that we decide a log-transform may be appropriate, we can use the tools available to us in python to also determine whether a simple log transform is *optimal* or if we can make improvments by changing the value of lambda.

- Would a Box_cox transform be appropriate for transforming price? Why or why not?
- What does the stats.boxcox function below do? What does the value of "ci" tell us about the value of l_lambda? 

In [ ]:
valiues, l_lambda, ci = stats.boxcox(data.price, alpha = .05)

## Compare the performance of a linear-log model against the linear-linear model using 5-fold cross validation. Use RMSE as the comparison metric.

Sklearn is the premier machine learning library within python and has many tools within for model fitting, tuning, testing, and evaluation. It has an implementation of OLS within it that is equivalent to the statsmodels implementation and is compatible with the other tools within sklearn that are not implemented within statsmodels (such as cross validation). 

Here is a reference table of metrics that can be used with the "scoring" hyperparameter and the general situations in which they would be appropriate:

| **Scoring Name**                         | **Description**                                                                                                                                     | **Applicable To**                |
|------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------|
| **accuracy**                             | Fraction of correct predictions ([`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)).         | Classification                   |
| **balanced_accuracy**                    | Accuracy adjusted for class imbalance.                                                                                                              | Classification                   |
| **average_precision**                    | Area under the precision-recall curve ([`average_precision_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html)). | Classification                   |
| **neg_brier_score**                      | Negative Brier score loss (the lower the better).                                                                                                   | Probabilistic Classification     |
| **f1**                                   | F1 score (harmonic mean of precision and recall).                                                                                                   | Binary Classification            |
| **f1_macro**                             | F1 score (macro-average).                                                                                                                           | Multiclass Classification        |
| **f1_micro**                             | F1 score (micro-average).                                                                                                                           | Multiclass Classification        |
| **f1_weighted**                          | F1 score (weighted average).                                                                                                                        | Multiclass Classification        |
| **f1_samples**                           | F1 score (by sample).                                                                                                                               | Multi-label Classification       |
| **neg_log_loss**                         | Negative log-likelihood loss (the lower the better).                                                                                                | Probabilistic Classification     |
| **precision**                            | Precision score.                                                                                                                                    | Binary Classification            |
| **precision_macro**                      | Precision (macro-average).                                                                                                                          | Multiclass Classification        |
| **precision_micro**                      | Precision (micro-average).                                                                                                                          | Multiclass Classification        |
| **precision_weighted**                   | Precision (weighted average).                                                                                                                       | Multiclass Classification        |
| **precision_samples**                    | Precision (by sample).                                                                                                                              | Multi-label Classification       |
| **recall**                               | Recall score.                                                                                                                                       | Binary Classification            |
| **recall_macro**                         | Recall (macro-average).                                                                                                                             | Multiclass Classification        |
| **recall_micro**                         | Recall (micro-average).                                                                                                                             | Multiclass Classification        |
| **recall_weighted**                      | Recall (weighted average).                                                                                                                          | Multiclass Classification        |
| **recall_samples**                       | Recall (by sample).                                                                                                                                 | Multi-label Classification       |
| **jaccard**                              | Jaccard similarity coefficient score.                                                                                                               | Binary Classification            |
| **jaccard_macro**                        | Jaccard score (macro-average).                                                                                                                      | Multiclass Classification        |
| **jaccard_micro**                        | Jaccard score (micro-average).                                                                                                                      | Multiclass Classification        |
| **jaccard_weighted**                     | Jaccard score (weighted average).                                                                                                                   | Multiclass Classification        |
| **jaccard_samples**                      | Jaccard score (by sample).                                                                                                                          | Multi-label Classification       |
| **roc_auc**                              | Area under the ROC curve ([`roc_auc_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)).                  | Binary Classification            |
| **roc_auc_ovr**                          | ROC AUC with one-vs-rest multiclass strategy.                                                                                                       | Multiclass Classification        |
| **roc_auc_ovo**                          | ROC AUC with one-vs-one multiclass strategy.                                                                                                        | Multiclass Classification        |
| **roc_auc_ovr_weighted**                 | Weighted average of ROC AUC OVR.                                                                                                                    | Multiclass Classification        |
| **roc_auc_ovo_weighted**                 | Weighted average of ROC AUC OVO.                                                                                                                    | Multiclass Classification        |
| **explained_variance**                   | Explained variance regression score.                                                                                                                | Regression                       |
| **max_error**                            | Maximum residual error.                                                                                                                             | Regression                       |
| **neg_mean_absolute_error**              | Negative mean absolute error.                                                                                                                       | Regression                       |
| **neg_mean_squared_error**               | Negative mean squared error.                                                                                                                        | Regression                       |
| **neg_root_mean_squared_error**          | Negative root mean squared error.                                                                                                                   | Regression                       |
| **neg_mean_squared_log_error**           | Negative mean squared logarithmic error.                                                                                                            | Regression                       |
| **neg_median_absolute_error**            | Negative median absolute error.                                                                                                                     | Regression                       |
| **r2**                                   | R<sup>2</sup> (coefficient of determination) regression score function.                                                                             | Regression                       |
| **neg_mean_poisson_deviance**            | Negative mean Poisson deviance.                                                                                                                     | Regression (count data)          |
| **neg_mean_gamma_deviance**              | Negative mean Gamma deviance.                                                                                                                       | Regression (positive data)       |
| **neg_mean_absolute_percentage_error**   | Negative mean absolute percentage error.                                                                                                            | Regression                       |
| **adjusted_mutual_info_score**           | Adjusted mutual information score.                                                                                                                  | Clustering                       |
| **adjusted_rand_score**                  | Adjusted Rand index.                                                                                                                                | Clustering                       |
| **completeness_score**                   | Completeness score.                                                                                                                                 | Clustering                       |
| **fowlkes_mallows_score**                | Fowlkes-Mallows index.                                                                                                                              | Clustering                       |
| **homogeneity_score**                    | Homogeneity score.                                                                                                                                  | Clustering                       |
| **mutual_info_score**                    | Mutual information score.                                                                                                                           | Clustering                       |
| **normalized_mutual_info_score**         | Normalized mutual information score.                                                                                                                | Clustering                       |
| **v_measure_score**                      | V-measure (harmonic mean of homogeneity and completeness).                                                                                          | Clustering                       |
| **rand_score**                           | Rand index adjusted for chance.                                                                                                                     | Clustering                       |

**Notes:**

- For regression metrics prefixed with `neg_`, the negative sign indicates that the metric is reversed in sign so that higher values are better (since cross-validation aims to **maximize** the score). For example, `neg_mean_squared_error` returns the negative MSE, so maximizing the negative MSE minimizes the MSE.
- Some metrics are suitable only for binary classification, while others can handle multiclass or multi-label problems.
- The `roc_auc` metric by default is for binary classification. For multiclass classification, you can use `roc_auc_ovr` (one-vs-rest) or `roc_auc_ovo` (one-vs-one).


In [ ]:
# sklearn functions work best with their own implementations of models
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [ ]:
# X must be a Dataframe and y is A Series 
X = data[['sqrft']]
Y = data['price']

model = LinearRegression()

scores = cross_val_score(model, X = X, y = Y,
                        scoring = ___, # Preferred metric
                         cv = ___) # Number of folds

# here you need to complete the code to take the mean and convert to a postive value
score

In [ ]:
# X must be a Dataframe and y is A Series 
X = ___
Y = data['price']

model = LinearRegression()

scores = cross_val_score(model, X = X, y = Y,
                        scoring = ___, # Preferred metric
                         cv = ___) # Number of folds

# here you need to complete the code to take the mean and convert to a postive value
score

## Question 8: Let's suppose we would like to add an additional regressor to our model "colonial".

Please fit the following regression in statsmodels:

$$price = \beta_1 + \beta_2 sqrft + \beta_3 sqrft^2+ \beta_3 colonial + e$$

- What is special about the variable "colonial"?
- What is the interpretation of the coefficient on "colonial"?

In [ ]:
model = smf.ols('___ ~ ___', data).fit()

model.summary()

## Question 8: Examine the following code where we plot the effect of sqrft on price and aswer the following:

- Why do we fix the value of colonial at one value?
- Why would we fix the value at the "mode"?
- If we added another variable (such as bedrooms or lotsize), what value may make sense for us to fix them at?

In [ ]:
# create a range of values for sqrft over which the model can make a prediction
prediction_values = pd.DataFrame(np.linspace(data['sqrft'].min(), data['sqrft'].max(), 1000), columns = ['sqrft'])

# below I add a fixed value for colonial
prediction_values['colonial'] = data['colonial'].mode().values[0]
y = model.predict(prediction_values)

prediction_values

In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(prediction_values['sqrft'], y, color = 'black')
plt.scatter(data['sqrft'], data['price'])
plt.ylabel('price')
plt.xlabel('sqrft')
plt.title('Sqrft Effect Plot')